# TVOJE SANJSKO MESTO
### Prijekt pri programiranju 1

In [5]:
import re

In [4]:
import pandas

In [3]:
import requests

In [6]:
def zajemi_prvo():
    '''Funkcija zajame besedilo prve spletne strani.'''
    naslov = 'http://www.weatherbase.com/weather/country.php3?r=EUR&regionname=Europe'
    r = requests.get(naslov)
    return r.text

vsebina_prve = zajemi_prvo()
print(vsebina_prve)

<!DOCTYPE html>
<html>
<head>
<META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=ISO-8859-1">
<link rel="canonical" href="http://www.weatherbase.com/weather/country.php3?r=EUR&regionname=Europe"/>
<meta name="keywords" content="Europe, travel, vacation, holiday, flight, hotel, weather data, historical weather data, climatology records, weather averages, weather normals, climate weather, weather database">
<meta name="description" content="Travel and vacation weather information for your holiday journey to Europe.">
<title>Europe Travel Weather Averages (Weatherbase)</title>
<meta property="og:title" content="Europe Travel Weather Averages (Weatherbase)"/>
<meta property="og:site_name" content="Weatherbase"/>
<meta property="og:type" content="website"/>
<meta property="og:url" content="http://www.weatherbase.com/weather/country.php3?r=EUR&regionname=Europe"/>
<meta property="og:image" content="http://www.weatherbase.com/gr/weatherbase-large-icon.png"/>
<link rel="stylesheet" hr

In [7]:
def najdi_drzave(besedilo):
    '''Sprejme vsebino spletne strani in poišče imena vseh evropskih drzav. 
    Zapise jih v obliki parametrov ki so v URLju nadalnih spletnih strani.'''
    seznam_drzav = list()
    seznam_kratic = list()
    seznam_parametrov = list()
    iskani_niz = r'<li><a class="redglow" href="/weather/\D*?.php3\?c=(?P<kratica>\D*?)&name=(?P<ime>\D*?)" onClick="recordOutboundLink'
    iscemo = re.compile(iskani_niz , flags=re.DOTALL)
    for drzava in re.finditer(iscemo, besedilo):
        seznam_drzav += [drzava.group('ime')]
        seznam_kratic += [drzava.group('kratica')]
        seznam_parametrov += ['{}&name={}'.format(drzava.group('kratica'),drzava.group('ime'))]
    return seznam_parametrov

parametri = najdi_drzave(vsebina_prve)
print(parametri)

['AL&name=Albania', 'AD&name=Andorra', 'AT&name=Austria', 'BY&name=Belarus', 'BE&name=Belgium', 'BA&name=Bosnia-and-Herzegovina', 'BG&name=Bulgaria', 'XJ&name=Channel-Islands', 'HR&name=Croatia', 'CY&name=Cyprus', 'CZ&name=Czech-Republic', 'DK&name=Denmark', 'EE&name=Estonia', 'FO&name=Faroe-Islands', 'FI&name=Finland', 'FR&name=France', 'DE&name=Germany', 'GI&name=Gibraltar', 'GR&name=Greece', 'VA&name=Holy-See', 'HU&name=Hungary', 'IS&name=Iceland', 'IE&name=Ireland', 'IM&name=Isle-of-Man', 'IT&name=Italy', 'XK&name=Kosovo', 'LV&name=Latvia', 'LI&name=Liechtenstein', 'LT&name=Lithuania', 'LU&name=Luxembourg', 'MK&name=Macedonia', 'MT&name=Malta', 'MD&name=Moldova', 'MC&name=Monaco', 'ME&name=Montenegro', 'NL&name=Netherlands', 'NO&name=Norway', 'PL&name=Poland', 'PT&name=Portugal', 'RO&name=Romania', 'RU&name=Russia', 'SM&name=San-Marino', 'RS&name=Serbia', 'SK&name=Slovakia', 'SI&name=Slovenia', 'ES&name=Spain', 'SE&name=Sweden', 'CH&name=Switzerland', 'TR&name=Turkey', 'UA&name=Ukr

In [8]:
def najdi_mesta(besedilo):
    '''Sprejme vsebino spletne strani in pojisce imena in sifro mest.
    Zapiše jih v obliki parametrov za URLje podstrani.'''
    seznam_parametrov = list()
    iskani_niz = r'<li><a class="redglow" href="/weather/weather.php3\?s=(?P<sifra>\d*?)&cityname=(?P<ime>\D*?)" onClick="recordOutboundLink'
    iscemo = re.compile(iskani_niz , flags=re.DOTALL)
    for najdeni in re.finditer(iscemo, besedilo):
        seznam_parametrov += ['s={}&cityname={}'.format(najdeni.group('sifra'),najdeni.group('ime') )]
    return seznam_parametrov

In [18]:
def zajemi_drugo(seznam):
    "Sprejme seznam parametrov v URLju spletnih strani, ki jih zajame in v njih zopet pojisce parametre nadalnjih podstrani."
    koncni_parametri = list()
    for parameter in seznam:
        naslov = r'http://www.weatherbase.com/weather/city.php3?c=' + parameter
        r = requests.get(naslov)
        koncni_parametri += najdi_mesta(r.text)
    return koncni_parametri

# Ne dela za United Kingdom!!!
vsi_parametri = zajemi_drugo(parametri)
print(vsi_parametri)

['s=136290&cityname=Barc-Albania', 's=136000&cityname=Boric-i-Math-Albania', 's=136150&cityname=Ceredzias-Albania', 's=11631&cityname=Durres-Albania', 's=592653&cityname=Fier-Albania', 's=52631&cityname=Gjinokaster-Albania', 's=92631&cityname=Korce-Albania', 's=592655&cityname=Likaj-Albania', 's=631&cityname=Shkoder-Albania', 's=51631&cityname=Tirane-Albania', 's=136220&cityname=Vlore-Albania', 's=342955&cityname=Andorra-La-Vella-Andorra', 's=592658&cityname=Soldeu-Andorra', 's=112341&cityname=Abriach-Austria', 's=591698&cityname=Abtenau-Austria', 's=591699&cityname=Admont-Austria', 's=591700&cityname=Aflenz-Kurort-Austria', 's=75111&cityname=Aigen-Austria', 's=591701&cityname=Allerheiligen-bei-Wildon-Austria', 's=591702&cityname=Amstetten-Austria', 's=111460&cityname=Apriach-Austria', 's=591704&cityname=Aschach-an-der-Steyr-Austria', 's=591705&cityname=Aspach-Austria', 's=591706&cityname=Aspang-Markt-Austria', 's=591707&cityname=Bad-Aussee-Austria', 's=591708&cityname=Bad-Bleiberg-Aus